# CASA0007 - Quantitative Methods Group Project
## Group 25

In [4]:
#Reading in libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Context of the data

### 1. Cycle to work - Joe
This dataset is from the 2011 Census. Results are based off a Census question that classifies usual residents aged 16 to 74 in England and Wales by their method of travel to work, disaggregated by Census Output Area (OA) and Rural/Urban classification. The estimates are as at census day, 27 March 2011 (single time snapshot). The dataset can be found [here](https://www.nomisweb.co.uk/census/2011/qs701ew).

### 2. Deprivation

### 3. Air Quality - Zach
[Data](https://data.london.gov.uk/dataset/pm2-5-map-and-exposure-data) linked here.


This dataset identifies modelled PM<sub>2.5 </sub>(particulate matter of diameter 2.5 micrometers or less) across London's Output Areas.

Increased exposure to high levels of partculate matter have been linked to increased rate of cardiovascular and respiratory disease (Liu, 2019). 

### 4. Cycle Sharing

## 2. Description of data fields/ variables

### 1. Cycle to work - Joe
* Read in the data

In [24]:
cycletowork = pd.read_csv('data/cycleing_to_work_oa.csv')
cycletowork.columns

Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Method of Travel to Work: All categories: Method of travel to work; measures: Value',
       'Method of Travel to Work: Work mainly at or from home; measures: Value',
       'Method of Travel to Work: Underground, metro, light rail, tram; measures: Value',
       'Method of Travel to Work: Train; measures: Value',
       'Method of Travel to Work: Bus, minibus or coach; measures: Value',
       'Method of Travel to Work: Taxi; measures: Value',
       'Method of Travel to Work: Motorcycle, scooter or moped; measures: Value',
       'Method of Travel to Work: Driving a car or van; measures: Value',
       'Method of Travel to Work: Passenger in a car or van; measures: Value',
       'Method of Travel to Work: Bicycle; measures: Value',
       'Method of Travel to Work: On foot; measures: Value',
       'Method of Travel to Work: Other method of travel to work; measures: Value',
       'Method of Travel to Work: Not in 

* Drop unnecessary columns

In [25]:
cycletowork = cycletowork[['date',
                           'geography',
                           'geography code',
                           'Rural Urban',
                           'Method of Travel to Work: Bicycle; measures: Value',
                           'Method of Travel to Work: All categories: Method of travel to work; measures: Value']]
cycletowork.columns

Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Method of Travel to Work: Bicycle; measures: Value',
       'Method of Travel to Work: All categories: Method of travel to work; measures: Value'],
      dtype='object')

### 2. Deprivation

### 3. Air Quality - Zach

In [9]:
aq_df = pd.read_csv("data/OA2_PM25_2013.csv")

### 4. Cycle Sharing

## 3. Summary statistic fields

## 4. Appropriate plots to communicate distribution

## 5. Appropriate plots to illustrate relationship between key fields

# References

Liu, C., Chen, R., Sera, F., Vicedo-Cabrera, A.M., Guo, Y., Tong, S., Coelho, M.S., Saldiva, P.H., Lavigne, E., Matus, P. and Valdes Ortega, N., 2019. Ambient particulate air pollution and daily mortality in 652 cities. New England Journal of Medicine, 381(8), pp.705-715.